In [3]:
################## IMPORTS
import pygame
import sys
import random
import math
from numpy import pi, sin, cos, sign, arctan, arccos, array

################## INITIALIZATION AND START VALUES
pygame.init()
WIDTH, HEIGHT = 800, 600
window = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption("Object Simulation")

def f(x, dest):
    c1, c2, c3, m = 0.01, 1, 1, 1
    f0 = x[1]
    f1 = (1/m) * (c1 * (dest - x[0]))# + c2 * (x - x)**(-1) + c3 * (x - x)**(-1))
    return array([f0, f1])

def phi(y, dest, dt, f):
    k1 = f(y, dest)
    k2 = f(y + dt/2*k1, dest)
    k3 = f(y + dt/2*k2, dest)
    k4 = f(y + dt*k3, dest)
    return 1/6*(k1 + 2*k2 + 2*k3 + k4)

################## RUNGE KUTTA 4TH ORDER METHOD
def oneStepMethod(y0, dest, n):
    y_n = [array(y0)]
    h = 1/n
    for _ in range(n):
        y_n.append(y_n[-1] + h*phi(y_n[-1],dest, h, f))
    return y_n[-1]

################## MOVING OBJECTS CLASSES                
class CircleObject:
    def __init__(self, id, x, y, radius, color, vel_x, vel_y):
        self.id = id
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.vel_x = vel_x
        self.vel_y = vel_y
    
    def __ne__(self, other):
        return self.id != other.id
    
    def draw(self):
        pygame.draw.circle(window, self.color, (self.x, self.y), self.radius)

    def move(self):# checking if the object is colliding with the window borders, if so it inverts the orientation of the velocity
        # self.x, self.vel_x = oneStepMethod([self.x, self.vel_x], pairs[self.id][0], 10)
        # self.y, self.vel_y = oneStepMethod([self.y, self.vel_y], pairs[self.id][1], 10)
        self.x += self.vel_x
        self.y += self.vel_y

        if self.x <= self.radius:
            self.x = self.radius
        elif self.x + self.radius >= WIDTH:
            self.x = WIDTH - self.radius
        if self.y <= self.radius:
            self.y = self.radius
        elif self.y + self.radius >= HEIGHT:
            self.y = HEIGHT - self.radius
    
    def paintProximity(self, x, y):
        position = {}
        # i will create a circle around the object
        border_points = [(self.x + self.radius * cos(k), self.y + self.radius * sin(k)) for k in range(0, 360, 10)]
        for i, j in border_points:
            distance = math.hypot(x - i, y - j) # for each point in the window, calculate the distance to the point (x, y)
            position[(i, j)] = distance
        return position
    
    def pointMove(self, x, y):
        x = min(max(x, self.radius), WIDTH - self.radius)
        y = min(max(y, self.radius), HEIGHT - self.radius)
        self.vel_x = (x - self.x) / 100
        self.vel_y = (y - self.y) / 100
        
    def repulsion(self, objects):
        # self.vel_x, self.vel_y = 0, 0
        for obj in objects:
            if obj != self:
                dist_x = self.x - obj.x
                dist_y = self.y - obj.y
                if (math.hypot(dist_x, dist_y) < self.radius + obj.radius + 5):
                    if dist_x == 0:
                        dist_x = 0.0001
                    if dist_y == 0:
                        dist_y = 0.0001
                    self.vel_x += 2000 * sign(dist_x) / dist_x ** 2
                    self.vel_y += 2000 * sign(dist_y) / dist_y ** 2
        
    def directMove(self, x, y, objects = []):
        pygame.draw.circle(window, self.color, (int(x), int(y)), 5)
        if math.hypot(x - self.x, y - self.y) <= 7:
            self.x = x
            self.y = y
            self.vel_x = 0
            self.vel_y = 0
            return False
        else:
            position = self.paintProximity(x, y)
            cord = min(position, key=position.get)
            self.pointMove(cord[0],cord[1])
            # self.repulsion(objects)
            return True

################## STATIC OBJECTS CLASSES
class CircleStaticObject:
    def __init__(self, id, x, y, radius, color):
        self.id = id
        self.x = x
        self.y = y
        self.radius = radius
        self.color = color
        self.vel_x = 0
        self.vel_y = 0

    def draw(self):
        pygame.draw.circle(window, self.color, (self.x, self.y), self.radius)

    def move(self):
        if self.x <= self.radius:
            self.x = self.radius
        elif self.x + self.radius >= WIDTH:
            self.x = WIDTH - self.radius
        if self.y <= self.radius:
            self.y = self.radius
        elif self.y + self.radius >= HEIGHT:
            self.y = HEIGHT - self.radius

        self.x += self.vel_x
        self.y += self.vel_y
                
################## FUNCTIONS OF THE SYSTEM

def handle_events():
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()

def objectsPositions(objects, static_object, dest):
    positions = []
    dx, dy = dest[0] - static_object.x, dest[1] - static_object.y
    signX = sign(dx)
    signY = sign(dy)
    if dx == 0 and dy == 0:
        return [(static_object.x, static_object.y)]*len(objects)
    elif dx == 0:
        alpha = pi/2 * signY # 90 or 270 degrees
        signX = 1
    elif dy == 0:
        alpha = 0 # 0 or 180 degrees
        signY = 1
    else:
        alpha =  arctan(dy/dx)
    if len(objects) % 2 != 0:
        positions.append((static_object.x + signX * cos(alpha  + signY * pi) * (static_object.radius + objects[0].radius), static_object.y + signX * sin(alpha  + signY * pi) * (static_object.radius + objects[0].radius)))
        for l in range(1, len(objects)):
            sumR = static_object.radius + objects[0].radius # because we are assuming that every object has the same radius
            sumRx2Sq = 2*sumR**2
            bw = (2 * arccos((sumRx2Sq - objects[0].radius**2)/(sumRx2Sq))) * (1 if l % 2 == 0 else -1) # if l is even, the angle is negative (bw for between)
            angleAdd = ((l-1)//2 + 1) * bw
            if abs(angleAdd + bw/2) > pi:
                return positions
            positions.append((static_object.x + signX * cos(alpha  + signY * pi + angleAdd) * sumR, static_object.y + signX * sin(alpha  + signY * pi + angleAdd) * sumR))
    else:
        for l in range(len(objects)):
            sumR = static_object.radius + objects[0].radius
            sumRx2Sq = 2*sumR**2
            bw = (arccos((sumRx2Sq - objects[0].radius**2)/(sumRx2Sq))) * (1 if l % 2 == 0 else -1) # if l is even, the angle is negative (bw for between)
            angleAdd = ((l//2)*2 + 1) * bw
            if abs(angleAdd + bw) > pi:
                return positions
            positions.append((static_object.x + signX * cos(alpha  + signY * pi + angleAdd) * sumR, static_object.y + signX * sin(alpha  + signY * pi + angleAdd) * sumR))
    return positions

def objectsTo(objects, positions):
    pairs = {}
    candidates = list(range(len(objects)))
    for pos in positions:
        closeIndex = array([math.hypot(pos[0] - objects[n].x, pos[1] - objects[n].y) for n in candidates]).argmin() # returns the index of the closest object to pos
        pairs[candidates[closeIndex]] = pos
        candidates.pop(closeIndex)
    return pairs
                
        
################## CREATING OBJECTS AND MAIN LOOP
def main():
    ################## CREATING OBJECTS
    objects = []
    num_objects = 4
    radius = 30
    for id in range(num_objects):
        x,y = random.randint(60, WIDTH-60), random.randint(60, HEIGHT-60)
        color = (0, 0, 0)
        # vel_x = random.choice([3, 4])
        vel_x = 0
        # vel_y = random.choice([3, 4])
        vel_y = 0
        obj = CircleObject(id, x, y, radius, color, vel_x, vel_y)
        objects.append(obj)
        
    ################## CREATING STATIC OBJECT
    static_object_radius = 50
    static_object_color = (255, 0, 0)
    static_object_x = WIDTH // 2
    static_object_y = HEIGHT // 2
    static_object = CircleStaticObject(4, static_object_x, static_object_y, static_object_radius, static_object_color)
    
    ################## PROBLEM 1: STATIC OBJECT NEED TO BE MOVED TO A SPECIFIC POSITION
    # dest = (WIDTH - 100, 100)
    dest = (100, 100)
    # dest = (100, HEIGHT - 100)
    # dest = (WIDTH - 100, HEIGHT - 100)
    # dest = (100, HEIGHT // 2)
    dmCoords = objectsPositions(objects, static_object, dest)
    num_objects = len(dmCoords)
    objects = objects[:num_objects]
    pairs = objectsTo(objects, dmCoords)
    dmList = [True]*num_objects

    clock = pygame.time.Clock()
    ################## MAIN LOOP
    while True:
        handle_events()
        window.fill((255, 255, 255))
        if dmList.count(False) == num_objects:
            destVel_x = (dest[0] - static_object.x) / 100
            destVel_y = (dest[1] - static_object.y) / 100
            static_object.vel_x = destVel_x
            static_object.vel_y = destVel_y
            for obj in objects:
                obj.vel_x = destVel_x
                obj.vel_y = destVel_y
        else:
            index = dmList.index(True)
            dmList[index] = objects[index].directMove(*pairs[index], objects)
            # for obj in objects:
            #     obj.directMove(*pairs[obj.id], objects)
            
        static_object.move()
        static_object.draw()
        
        for ind in range(num_objects):
            # if dmList[ind]:
            #     dmList[ind] = objects[ind].directMove(*dmCoords[ind], objects)
            # obj.pointMove(pygame.mouse.get_pos()[0], pygame.mouse.get_pos()[1])
            objects[ind].move()
            objects[ind].draw()
        # pygame.draw.line(window, (0,0,0), dest, (static_object_x, static_object_y), 2)
        pygame.draw.circle(window, static_object.color, dest, 5)

        pygame.display.update()
        clock.tick(60)

if __name__ == "__main__":
    main()


SystemExit: 